In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json
with open('chatbot_vitfaculty.json') as file:
    data = json.load(file)


words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [5]:
words

['Hi',
 'How',
 'are',
 'you',
 'Is',
 'anyone',
 'there',
 '?',
 'Hello',
 'Good',
 'day',
 'Whats',
 'up',
 'cya',
 'See',
 'you',
 'later',
 'Goodbye',
 'I',
 'am',
 'Leaving',
 'Have',
 'a',
 'Good',
 'day',
 'Mam',
 'may',
 'I',
 'known',
 'your',
 'free',
 'time',
 '?',
 'when',
 'are',
 'you',
 'free',
 'mam',
 '?',
 'what',
 'are',
 'your',
 'free',
 'slots',
 'mam',
 '?',
 'when',
 'is',
 'your',
 'free',
 'time',
 'mam',
 '?',
 'when',
 'can',
 'we',
 'come',
 'for',
 'review',
 'mam',
 'when',
 'is',
 'the',
 'review',
 'mam',
 'Mam',
 'you',
 'have',
 'marked',
 'me',
 'absent',
 'today',
 'Mam',
 'i',
 'am',
 'debarred',
 'please',
 'give',
 'me',
 'attendance',
 'Mam',
 'can',
 'you',
 'please',
 'give',
 'me',
 'one',
 'class',
 'attendance',
 'when',
 'is',
 'the',
 'quiz',
 'mam',
 'what',
 'time',
 'is',
 'the',
 'quiz',
 'mam']

In [6]:
data['intents']

[{'context_set': '',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'tag': 'greeting'},
 {'context_set': '',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'tag': 'goodbye'},
 {'context_set': '',
  'patterns': ['Mam may I known your free time?',
   'when are you free mam?',
   'what are your free slots mam?',
   'when is your free time mam?'],
  'responses': ['I am free after 3',
   'I am not free today',
   "I am busy today let's meet tommorrow",
   'come after 5 p.m'],
  'tag': 'free time'},
 {'context_set': '',
  'patterns': ['when can we come for review mam', 'when is the review mam'],
  'responses': ['I am free after 3',
   'I am not free today',
   "I am busy today let's meet tommorrow",
   'come after 5 p.m']

In [0]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

In [8]:
words

['a',
 'abs',
 'am',
 'anyon',
 'ar',
 'attend',
 'can',
 'class',
 'com',
 'cya',
 'day',
 'debar',
 'for',
 'fre',
 'giv',
 'good',
 'goodby',
 'hav',
 'hello',
 'hi',
 'how',
 'i',
 'is',
 'known',
 'lat',
 'leav',
 'mam',
 'mark',
 'may',
 'me',
 'on',
 'pleas',
 'quiz',
 'review',
 'see',
 'slot',
 'the',
 'ther',
 'tim',
 'today',
 'up',
 'we',
 'what',
 'when',
 'yo',
 'you']

In [0]:
labels = sorted(labels)

In [10]:
labels

['Attendance', 'Review', 'free time', 'goodbye', 'greeting', 'quiz']

In [0]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

In [12]:
training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [14]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.56712 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.56712 - acc: 0.8932 -- iter: 16/22
Training Step: 3000  | total loss: 0.78266 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.78266 - acc: 0.8205 -- iter: 22/22
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [15]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: what about the review?
I am busy today let's meet tommorrow
You: ok mam
Hello!
You: when is the quiz?
postponed to next week
You: can you please give me attendance?
sorry I cannot help
You: mam you wrongly marked me absent
ok I will take care
You: thanks mam
Hi there, how can I help?
You: quit
